In [1]:
import torch
import torchvision.transforms as transforms
import torchvision.models as models
from torch.autograd import Variable
import cv2
import os
import pandas as pd
import numpy as np
from tqdm import tqdm

C:\Users\jaivi\anaconda3\envs\GPU_TF\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import pandas as pd

# Read the CSV file into a DataFrame
df1 = pd.read_csv(r"C:\Users\jaivi\Downloads\MovieData\Video_Data\movie_description.csv")

# # Replace "videos_1000\" with "/kaggle/input/movie-clips/" in the "video_path" column
# df1['video_path'] = df1['video_path'].str.replace(r'videos_1000\\', '/kaggle/input/movie-clips/', regex=True)

# Save the modified DataFrame back to a CSV file
df1.to_csv('modified_file.csv', index=False)

print("Replacement complete and saved to modified_file.csv")

Replacement complete and saved to modified_file.csv


In [ ]:
df1.count()

In [ ]:
from os import listdir

# Specify the directory path
directory = "/kaggle/input/movie-clips/"

# Get all files in the directory
files = listdir(directory)

# Count the number of files
num_files = len(files)

# Print the count
print("Number of files in the directory:", num_files)

In [ ]:
import os

# Replace "your_id_here" with the actual video ID
video_id_to_delete = "3365243"
directory = "/kaggle/input/movie-clips/"

# Get all video files in the directory
video_files = [f for f in os.listdir(directory) if f.endswith(".mp4")]

# Filter videos based on ID (without deleting)
filtered_videos = [f for f in video_files if f.split(".")[0] != video_id_to_delete]

# Print the filtered video list (for information)
print("Videos after filtering:", filtered_videos)

# Caution: This line permanently deletes the file (use with care)
os.remove(os.path.join(directory, video_id_to_delete + ".mp4"))


In [2]:
import pandas as pd

# Read the CSV file into a pandas DataFrame
df = pd.read_csv(r"C:\Users\jaivi\Downloads\MovieData\Video_Data\movie_description.csv")

# Filter out the entry with video_id "3365243"
filtered_df = df[df['video_id'] != 3365243]

# Save the filtered DataFrame to a new CSV file (optional)
filtered_df.to_csv('filtered_videos.csv', index=False)

# Print a confirmation message
print("Entry for video_id '3365243' removed successfully!")

Entry for video_id '3365243' removed successfully!


In [3]:
filtered_df.count()

video_id       999
video_path     999
description    999
dtype: int64

In [4]:
# Load the CSV file
df = pd.read_csv(r"C:\Users\jaivi\Downloads\MovieData\Video_Data\filtered_videos.csv")

In [5]:
df.count()

video_id       999
video_path     999
description    999
dtype: int64

In [6]:
# Check for GPU availability
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [7]:
# Load the pre-trained model and move it to the GPU
model = models.resnet50(pretrained=True)
model = model.eval().to(device)

In [8]:
# Define the image transformation
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [9]:
def extract_features(video_path, model, device, batch_size=16):
    cap = cv2.VideoCapture(video_path)
    features = []
    batch = []
    
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        frame = transform(frame)
        batch.append(frame)
        
        if len(batch) == batch_size:
            batch = torch.stack(batch).to(device)
            with torch.no_grad():
                batch_features = model(batch)
            features.append(batch_features.cpu().numpy())
            batch = []
    
    # Process remaining frames
    if batch:
        batch = torch.stack(batch).to(device)
        with torch.no_grad():
            batch_features = model(batch)
        features.append(batch_features.cpu().numpy())
    
    cap.release()
    
    features = np.concatenate(features, axis=0)
    return features.mean(axis=0)

In [10]:
# Dictionary to store the extracted features
video_features = {}

In [11]:
# Add a progress bar for the video processing loop
for idx, row in tqdm(df.iterrows(), total=len(df), desc="Processing videos"):
    video_id = row['video_id']
    video_path = row['video_path']
    try:
        features = extract_features(video_path, model, device)
        video_features[video_id] = features
    except Exception as e:
        print(f"Failed to process video {video_id}: {e}")

Processing videos:   0%|          | 0/999 [00:00<?, ?it/s]C:\Users\jaivi\anaconda3\envs\GPU_TF\lib\site-packages\torch\nn\functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ..\c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)
Processing videos:   0%|          | 4/999 [02:19<9:38:56, 34.91s/it] 


KeyboardInterrupt: 

In [ ]:
# Save the extracted features to a file
np.save('video_features_pytorch1.npy', video_features)

In [ ]:
import numpy as np

# Load the saved features
video_features = np.load('video_features_pytorch.npy', allow_pickle=True)

# Analyze feature details
print("Shape of the features:", video_features.shape)

# Assuming each video has one feature vector
num_videos, feature_size = video_features.shape

# Print data type of the features
print("Data type of features:", video_features.dtype)

# Print some example feature values
print("Example feature values (first 5 elements):", video_features[0][:5])

# Calculate basic statistics (optional)
mean_features = np.mean(video_features, axis=0)
std_features = np.std(video_features, axis=0)

# Print mean and standard deviation of features
print("Mean of features:", mean_features[:5])  # Print only first 5 for brevity
print("Standard deviation of features:", std_features[:5])

X_features = video_features.reshape(num_videos, feature_size)

# Now you have X_features which can be used to train a text description model
print("Shape of features for training:", X_features.shape)


Feature extraction  with feature to video ID mapping(Improved method)

In [ ]:
!git clone https://github.com/PyAV-Org/PyAV.git
%cd PyAV


# Either install the testing dependencies:
!pip install --upgrade -r tests/requirements.txt

# Build PyAV.
!make
!pip install .

In [ ]:
!make
!pip install .

In [3]:
import numpy as np
import pandas as pd
import os
import torch
import torchvision.transforms as transforms
from torchvision.models import resnet50
from torchvision.io import read_video
from PIL import Image
from tqdm import tqdm

# Load the CSV file
df = pd.read_csv('/kaggle/working/filtered_videos.csv')

# Initialize the pre-trained ResNet model
model = resnet50(pretrained=True)
model = torch.nn.Sequential(*list(model.children())[:-1])  # Remove the final classification layer
model.eval()

# Check for GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Function to extract features
def extract_features(video_path, model, batch_size=16):
    video, _, _ = read_video(video_path, pts_unit='sec')
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    
    frames = []
    for frame in video:
        # Convert tensor to PIL image
        frame = Image.fromarray(frame.numpy())
        frame = transform(frame).unsqueeze(0)
        frames.append(frame)
    
    frames = torch.cat(frames).to(device)
    
    features_list = []
    with torch.no_grad():
        for i in range(0, len(frames), batch_size):
            batch_frames = frames[i:i + batch_size]
            features = model(batch_frames).cpu()
            features_list.append(features)

    features = torch.cat(features_list)
    return features.mean(dim=0).numpy()

# Extract and save features
video_features = {}
missing_videos = 0

for idx, row in tqdm(df.iterrows(), total=df.shape[0], desc="Extracting features"):
    video_id = str(row['video_id'])
    video_path = row['video_path']
    if not os.path.exists(video_path):
        print(f"Video {video_path} not found.")
        missing_videos += 1
        continue
    try:
        features = extract_features(video_path, model)
        video_features[video_id] = features
    except Exception as e:
        print(f"Failed to extract features for {video_path}: {e}")
        missing_videos += 1

print(f"Total videos processed: {len(video_features)}")
print(f"Total missing videos: {missing_videos}")

# Save the features
np.save('video_features_pytorch_new.npy', video_features)

/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Extracting features:   0%|          | 2/999 [00:08<1:14:09,  4.46s/it]


KeyboardInterrupt: 